# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de dictionnaires : chaque phrase a une clé "mots" qui est associée à une liste des mots, et une clé "POS" associée à une liste de POS. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [1]:
def load_corpus(file):
    with open(file, "r") as f:
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_dico = {"mots" : [], "gold_labels" : []} # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                phrase_dico["mots"].append(features[1])
                phrase_dico["gold_labels"].append(features[3])
        corpus.append(phrase_dico)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [2]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[100], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

{'mots': ["L'", 'œuvre', 'est', 'située', 'dans', 'la', 'galerie', 'des', 'de', 'les', 'batailles', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.'], 'gold_labels': ['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT']}

{'mots': ['Ce', 'résultat', 'provient', 'à', 'hauteur', 'de', '18,5', 'ME', 'de', 'la', 'progression', 'de', 'la', 'rentabilité', 'de', "l'", 'habitat', 'de', 'loisirs', 'qui', 'dégage', 'une', 'marge', 'opérationnelle', 'de', '9,5', '%', 'contre', '8,5', '%', "l'", 'année', 'passée', '.'], 'gold_labels': ['DET', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'NUM', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'NUM', 'SYM', 'ADP', 'NUM', 'SYM', 'DET', 'NOUN', 'ADJ', 'PUNCT']}

{'mots': ['Cette', 'espèce', 'est', 'endémique', 'du', 'de', 'le', 'département'

## Extraction des caractéristiques

La fonction feature_extraction renvoie une liste de dictionnaires (un par mot) qui contiennent les caractéristiques suivantes pour chaque mot :
- mot
- mot précédent : pour le premier mot de la phrase son mot précédent sera "START", ce qui permettra de prendre en compte la caractéristique "être le premier mot".
- mot suivant : pour le dernier mot, ce sera "END".
- commence par une lettre majuscule
- est numérique
- contient des caractères non alphanumériques
- longueur du mot

Une fois que l'extraction des caractéristiques a été faite, on n'a plus besoin de conserver les phrases. Cela nous permettra de réellement mélanger tous les mots du corpus dans le perceptron.

In [5]:
def feature_extraction(corpus):
    
    # clés des dictionnaires
    mot = "mot"
    gold = "gold_label"
    mot_prec = "préc"
    mot_suiv = "suiv"
    maj = "maj"
    num = "nb"
    nonAlphanum = "non alphanum"
    long = "longueur"

    corpus_features = [] # copie de la liste pour ne pas modifier la liste d'origine

    for phrase in corpus: # ajout des features additionnelles
        for prev, word, suiv in zip(["START"] + phrase["mots"][:-1], phrase["mots"], phrase["mots"][1:] + ["END"]):
            # création de triplets (mot précédent, mot, mot suivant)
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier

            # ajout d'un dictionnaire de features du mot au corpus
            corpus_features.append({mot : word, 
                gold : phrase["gold_labels"][phrase["mots"].index(word)], # on récupère le gold_label correspondant
                mot_prec : prev,
                mot_suiv : suiv,
                num : word.isnumeric(),
                nonAlphanum : not word.isalnum(),
                long : len(word)})

    return corpus_features

gsd_train_features = feature_extraction(gsd_train)
gsd_dev_features = feature_extraction(gsd_dev)
gsd_test_features = feature_extraction(gsd_test)

In [9]:
print(*gsd_train_features[3:6], sep="\n")

{'mot': 'sont', 'gold_label': 'AUX', 'préc': 'cérébrales', 'suiv': 'devenu', 'nb': False, 'non alphanum': False, 'longueur': 4}
{'mot': 'devenu', 'gold_label': 'VERB', 'préc': 'sont', 'suiv': 'si', 'nb': False, 'non alphanum': False, 'longueur': 6}
{'mot': 'si', 'gold_label': 'ADV', 'préc': 'devenu', 'suiv': 'courantes', 'nb': False, 'non alphanum': False, 'longueur': 2}


## Implémentation de l'algorithme de classification
On a choisi d'implémenter la classification avec un perceptron moyenné.

Etiquettes du corpus : ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X

Sources : cours 3/12 + perceptron.pdf + zoom du 17/12

In [10]:
def perceptron_train(training_set, MAX_EPOCH=10):
    tags = ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]

    # TODO initialisation des poids : à zéros ou bien de manière aléatoire ??
    # je suis pas sûre du format de ces poids....
    a = [[0]*len(training_set[0]) for tag in tags] # poids moyennés

    for i in range(0, MAX_EPOCH):
        w = [[0]*len(training_set[0]) for tag in tags] # initialisé du vecteur de poids
        shuffled_set = shuffle(training_set.copy()) # on mélange le training set

        for phrase in shuffled_set:
            for word in phrase:
                prediction = max()
                # on cherche le tag le plus probable... mais comment ?... 
                # il va falloir donner une probabilité à chaque mot dis donc.... ???

                # on compare avec le gold_label
                # si mauvaise prédiction
                    #  on ajoute x(i) à chaque poids de l'étiquette correcte
                    #  on retire x(i) à chaque poids de l'étiquette mal prédite

        # on ajoute w à a
    return a

poids_gsd_train = perceptron_train(gsd_train_features)

## Evaluation des performances sur le corpus de validation (Dev)
Il faudra bien tester les hyper paramètres (epoch...)

## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine